In [1]:
import lightkurve as lk
import pandas as pd
import phasefolder
import os

%load_ext autoreload
%autoreload 2

In [23]:
def csv_to_lc(file):
    lcdata = pd.read_csv(file)
    lc = lk.LightCurve(time=lcdata.time, 
                       flux=lcdata.flux,
                       flux_err=lcdata.flux_err,
                       quality=lcdata.quality,
                       label=file.split("/")[-1].split(".csv")[0])
    return lc

In [24]:
lcc = [csv_to_lc("lightcurves/"+file) for file in os.listdir("lightcurves")]

In [64]:
from ipywidgets import interactive, widgets
import matplotlib.pyplot as plt

# elements
## index in lcc
a = widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(lcc)-1,
    step=1,
    description='Index:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
## period to fold on
b = widgets.BoundedFloatText(
    value=5,
    min=0.1,
    step=0.005,
    description='Period:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
## halve the period
half_button = widgets.Button(
    description=f"period /2",
    disabled=False,
    button_style='',
    tooltip="period/2",
)
half_button.factor = 0.5  # giving these buttons a new attribute 'factor'
## double the period
double_button = widgets.Button(
    description=f"period x2",
    disabled=False,
    button_style='',
    tooltip="2*period",
)
double_button.factor = 2
## triple the period
triple_button = widgets.Button(
    description=f"period x3",
    disabled=False,
    button_style='',
    tooltip="3*period",
)
triple_button.factor = 3
## change the center of the phase folded plot
time_min = lcc[0].time[lcc[0].flux==lcc[0].flux.min()].value[0]
c = widgets.BoundedFloatText(
    value=time_min,
    min=min(lcc[a.value].time.value),
    max=max(lcc[a.value].time.value),
    step=1,
    description='Epoch Time:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
## save the plot and the folded light curve data
save = widgets.Button(
    description='Save',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save',
)

# behaviors
def f(index, period, center):
    """Updates plot on index, period, or center point changes
    """
    lc = lcc[index]
    folded_lc = lc.fold(period, normalize_phase=False, epoch_time=center)

    fig, ax = plt.subplots(2, figsize=(10,4), tight_layout=True)
    lc.scatter(ax=ax[0])
    folded_lc.scatter(ax=ax[1], label=f"Period: {period:.3f} d");
    plt.title(lc.label)
    plt.show()
    return fig


def update_best_guess(*args):
    """Updates the period when the index changes
    
    Must be seperate from f() because f changes when the period
    does, so b would constantly get changed back
    """
    lc = lcc[a.value]
    best_period = phasefolder.find_period(lc)
    b.value = best_period


def factor_button(button):
    """Multiplies the period by the factor of the button
    """
    b.value = float(button.factor*b.value)


def on_save_clicked(button):
    """Saves the plot and the folded light curve data
    """
    label = lcc[a.value].label
    fig = f(a.value, b.value, c.value)
    fig.savefig(f"{label}.png")
    folded_lc = lc.fold(b.value, normalize_phase=False)
    folded_lc.to_csv(f"{label}.csv", overwrite=True);

# linking
a.observe(update_best_guess, 'value')
half_button.on_click(factor_button)
double_button.on_click(factor_button)
triple_button.on_click(factor_button)
save.on_click(on_save_clicked)

# layout
factors = widgets.HBox(
    [half_button, double_button, triple_button],
    layout=widgets.Layout(width='50%')
)
out = widgets.interactive_output(f, {'index': a, 'period': b, 'center': c})
# initialize and display
update_best_guess()
display(a, b, factors, c, out, button)

BoundedIntText(value=0, description='Index:', layout=Layout(width='50%'), max=9)

BoundedFloatText(value=4.368198550391359, description='Period:', layout=Layout(width='50%'), min=0.1, step=0.0…

BoundedFloatText(value=1341.7865923252236, description='Epoch Time:', layout=Layout(width='50%'), max=1353.161…

Output()

Button(description='Save', style=ButtonStyle(), tooltip='Save')